In [1]:
import googleapiclient.discovery
import pickle, json
from collections import defaultdict
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [2]:
ApiKey = open("youtube-API-key.txt").read().strip()

In [3]:
# https://developers.google.com/youtube/v3/docs

In [5]:
api_service_name = "youtube"
api_version = "v3"

RES = []

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=ApiKey)
request = youtube.search().list(
        part="snippet",
        maxResults=50,
        q="climate change"
    )

while len(RES) < 200:
    res = request.execute()
    RES.extend(res['items'])
    request = youtube.search().list_next(request, res)
pickle.dump(RES,open("video-list.pickle","wb"))
len(RES)

200

In [6]:
for xx in RES[:5]:
    print(xx['snippet']['title'])
RES[0]

Major milestones in battling climate change seen in 2022
CBS slammed for bizarre claim about climate change
What is Climate Change? | Start Here
We WILL Fix Climate Change!
What is Climate Change?: Crash Course Climate &amp; Energy #1


{'kind': 'youtube#searchResult',
 'etag': '3mOzdbyfPl86pSOLCtNULOeI_wQ',
 'id': {'kind': 'youtube#video', 'videoId': 'N7Qot_aax9M'},
 'snippet': {'publishedAt': '2022-12-20T16:52:53Z',
  'channelId': 'UCBi2mrWuNuyYy4gbM6fU18Q',
  'title': 'Major milestones in battling climate change seen in 2022',
  'description': 'While saving the climate may be decades away, 2022 saw some major advancements in the battle against climate change.',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/N7Qot_aax9M/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/N7Qot_aax9M/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/N7Qot_aax9M/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'ABC News',
  'liveBroadcastContent': 'none',
  'publishTime': '2022-12-20T16:52:53Z'}}

In [11]:
vid = RES[0]['id']['videoId']
# vid = "N6t6QHQtdVw"
vid

'N7Qot_aax9M'

In [13]:
def getVideoComments(vid):
    request = youtube.commentThreads().list(
        part="snippet,replies",
        maxResults=100,
        videoId=vid
    )
    try:
        response = request.execute()
    except googleapiclient.http.HttpError as ee:
        if b'"reason": "commentsDisabled"' in ee.args[1]:
            return None
        else:
            raise
    print(response['pageInfo'], len(response['items']), response.get("nextPageToken",">>> No nextPageToken!")[:20])
    json.dump(response,open(f"comments-data/{vid}.json","w"))
    return response
response = getVideoComments(vid)

{'totalResults': 19, 'resultsPerPage': 100} 19 >>> No nextPageToken


In [14]:
def myPrint(*args):
    pass
    # print(*args)

def printComment(ii, item, level=""):
    global tmp
    msg = level
    msg += f'{ii:3}: '
    snippet = item['snippet']
    STATS.nrComments += 1
    if 'topLevelComment' in snippet:
        snippet = snippet['topLevelComment']['snippet']
    STATS.authors[snippet['authorDisplayName']] += 1
    msg += snippet['textDisplay'][:125] + " (" + snippet['authorDisplayName'] + ")"
    myPrint(msg)
    replyList = item.get('replies',{}).get('comments',[])
    if replyList:
        STATS.nrThreads += 1
    for nr,reply in enumerate(replyList):
        tmp = reply
        STATS.nrReplies += 1
        printComment(nr, reply, level + "     ")
    nrReplies = int(item['snippet'].get('totalReplyCount',0))
    STATS.maxThreadLength = max(STATS.maxThreadLength, nrReplies)
    if nrReplies > len(replyList):
        myPrint(level,"     ","+++")

def analyseVideo(response):
    global STATS
    STATS = AttrDict(nrComments=0, nrThreads=0, nrReplies=0, maxThreadLength=0, authors=defaultdict(int), disabled=True)
    if response is not None:
        STATS.disabled = False
        for ii,xx in enumerate(response['items']):
            printComment(ii,xx)
    AUTHORS = STATS.authors
    del STATS.authors
    STATS.nrAuthors = len(AUTHORS)
    STATS.nrAuthors3Posts = len([xx for xx in AUTHORS if AUTHORS[xx]>2])
    STATS.maxPostsByAuthor = max([0] + list(AUTHORS.values()))
    return STATS

analyseVideo(response)

{'nrComments': 32,
 'nrThreads': 5,
 'nrReplies': 13,
 'maxThreadLength': 8,
 'disabled': False,
 'nrAuthors': 19,
 'nrAuthors3Posts': 3,
 'maxPostsByAuthor': 6}

In [ ]:
output = {}
for vv in RES:
    vid = vv['id'].get('videoId',None)
    if vid is None or vid in output:
        continue
    response = getVideoComments(vid)
    stats = analyseVideo(response)
    output[vid] = stats

{'totalResults': 19, 'resultsPerPage': 100} 19 >>> No nextPageToken
{'totalResults': 100, 'resultsPerPage': 100} 100 QURTSl9pM2tNQlpOeEhI
{'totalResults': 100, 'resultsPerPage': 100} 100 QURTSl9pMlhFNmo3RnJ2
{'totalResults': 100, 'resultsPerPage': 100} 100 QURTSl9pM0pKdWF3WU9v
{'totalResults': 84, 'resultsPerPage': 100} 84 >>> No nextPageToken
{'totalResults': 100, 'resultsPerPage': 100} 100 QURTSl9pMXJjNVZQOXNm
{'totalResults': 100, 'resultsPerPage': 100} 100 QURTSl9pMC1rRGYxTFFp
{'totalResults': 97, 'resultsPerPage': 100} 97 QURTSl9pMnhLeGxxcHpy
{'totalResults': 100, 'resultsPerPage': 100} 100 QURTSl9pMnVSeTRmeGNV
{'totalResults': 64, 'resultsPerPage': 100} 64 >>> No nextPageToken
{'totalResults': 89, 'resultsPerPage': 100} 89 >>> No nextPageToken
{'totalResults': 100, 'resultsPerPage': 100} 100 QURTSl9pM3BtNUg3Nkc0
{'totalResults': 100, 'resultsPerPage': 100} 100 QURTSl9pMFlvVXJ1Wnot
{'totalResults': 36, 'resultsPerPage': 100} 36 >>> No nextPageToken
{'totalResults': 99, 'resultsPer

In [16]:
vv

{'kind': 'youtube#searchResult',
 'etag': 'OlowAFreWVuR3tAJOhi2OVPmPXI',
 'id': {'kind': 'youtube#channel', 'channelId': 'UCPbyIYGQv3BQebWiC9mKOBA'},
 'snippet': {'publishedAt': '2020-06-02T17:40:02Z',
  'channelId': 'UCPbyIYGQv3BQebWiC9mKOBA',
  'title': 'Climate Сhange',
  'description': "Hello friends. The climate is changing and that's a fact. People are increasingly confronted with the incredible power of nature.",
  'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/OBq7B8KhOgTBj4ssL9FFpBrp1Fe75ZIM3dL--cavHsJKhsp3AREkfkfiPKJ0E0WsC02XMe0TAw=s88-c-k-c0xffffffff-no-rj-mo'},
   'medium': {'url': 'https://yt3.ggpht.com/OBq7B8KhOgTBj4ssL9FFpBrp1Fe75ZIM3dL--cavHsJKhsp3AREkfkfiPKJ0E0WsC02XMe0TAw=s240-c-k-c0xffffffff-no-rj-mo'},
   'high': {'url': 'https://yt3.ggpht.com/OBq7B8KhOgTBj4ssL9FFpBrp1Fe75ZIM3dL--cavHsJKhsp3AREkfkfiPKJ0E0WsC02XMe0TAw=s800-c-k-c0xffffffff-no-rj-mo'}},
  'channelTitle': 'Climate Сhange',
  'liveBroadcastContent': 'upcoming',
  'publishTime': '2020-06-02T17:4

In [215]:
len(RES), len(output)

(200, 81)

In [222]:
for vid,info in output.items():
    info['url'] = 'https://www.youtube.com/watch?v=' + vid

In [224]:
import pandas as pd
df = pd.DataFrame.from_dict(output,orient="index")
df.to_excel("result.xlsx")
df

,nrComments,nrThreads,nrReplies,maxThreadLength,disabled,nrAuthors,nrAuthors3Posts,maxPostsByAuthor,url
N7Qot_aax9M,32,5,13,8,False,19,3,6,https://www.youtube.com/watch?v=N7Qot_aax9M
_evqapQpxUQ,105,3,5,2,False,49,4,18,https://www.youtube.com/watch?v=_evqapQpxUQ
dcBXmj1nMTQ,120,13,20,7,False,98,4,7,https://www.youtube.com/watch?v=dcBXmj1nMTQ
9PFhrpyWV-w,110,8,26,9,False,99,1,3,https://www.youtube.com/watch?v=9PFhrpyWV-w
qySBQjSXbfw,123,11,23,8,False,103,5,5,https://www.youtube.com/watch?v=qySBQjSXbfw
...,...,...,...,...,...,...,...,...,...
Ggp5dpGQKZA,114,7,14,3,False,101,3,3,https://www.youtube.com/watch?v=Ggp5dpGQKZA
69DFis2WgMQ,121,9,21,48,False,106,3,3,https://www.youtube.com/watch?v=69DFis2WgMQ
A0Pvk3hwt7Y,48,15,20,4,False,31,1,13,https://www.youtube.com/watch?v=A0Pvk3hwt7Y
Ntg1ANdeYLg,140,19,40,17,False,103,6,10,https://www.youtube.com/watch?v=Ntg1ANdeYLg
